In [ ]:
#Import all libraries
import pandas as pd
from datetime import datetime,timedelta,date
import time
import urllib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc 
from decimal import Decimal

In [ ]:
# Use read_CSV() to extract data from John Hopkins GITHUB
today = date.today()
yesterday = today - timedelta(days=1)
date_Converted = yesterday.strftime("%m-%d-%Y")
try:
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/" + date_Converted + ".csv"
    df = pd.read_csv(url)
except urllib.error.HTTPError:
    f = open("C:\\Users\\JosephGaiennie\\Desktop\\Python\\Corona_Logs.txt", "a")
    f.write(str(time.time()) +  " " + "HTTPError")
    f.close()
df = df.fillna(0)


In [ ]:
# Graph confirmed cases against each State and Save the immage
df.plot(kind='bar',x='Province_State',y='Confirmed')
fig = plt.gcf()
fig.set_size_inches(20, 10.5)
plt.xticks(fontsize=14)
y = [min(df["Confirmed"]),max(df["Confirmed"])]
plt.yticks(np.arange(y[0], y[1], 10000))
plt.savefig("C:\\Users\\JosephGaiennie\\Desktop\\Python\\CoronaVirusPics\\USCoronaVirus" + date_Converted + ".png")

In [ ]:
# Use pyodbc library to make a connection to sql server database hosted on AWS using RDS
server = 'practicedatabase.c8pv3jjiflii.us-east-2.rds.amazonaws.com' 
database = 'CoronaDB' 
#username = '' Hidden
#password = '' Hidden
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [ ]:
#conn = pyodbc.connect('Driver={SQL Server Native Client 11.0};'
#                      'Server=LAPTOP-D3KCPF8R;'
#                      'Database=Practice_DB;'
#                      'Trusted_Connection=yes;')

In [ ]:
for index, row in df.iterrows():
    cursor.execute(
        """INSERT INTO [dbo].[COVID19_Analysis] ([State]
           ,[Date]
           ,[Deaths]
           ,[Recovered]
           ,[Incident_Rate]
           ,[People_Tested]
           ,[People_Hospitilized]
           ,[Moralitiy_Rate]
           ,[Confirmed]) VALUES(?,?,?,?,?,?,?,?,?)
           """,
                   (row['Province_State'],date_Converted,row['Deaths'],row['Recovered'],(row['Incident_Rate']),row['People_Tested'],int(row['People_Hospitalized']),row['Mortality_Rate'],row['Confirmed']))
    conn.commit()
conn.close()